In [67]:
import requests
import json
import pandas as pd
import geopandas
import numpy as np

def get_responses(*args):
    list_of_responses={}
    for parm in args:
        url = f"https://emotional.byteroad.net/collections/{parm}/items?f=json&lang=en-US&limit=10000&skipGeometry=false&offset=0"
        response = requests.get(url)
        
        if response.status_code == 200:
            data_dict = json.loads(response.content)
            list_of_responses[parm]=data_dict
        else:
            print("Error: API request unsuccessful.")
    return list_of_responses

In [68]:
response = get_responses('london_whitechapel_sub-oe309002')['london_whitechapel_sub-oe309002']['features']
selected_columns = ['time', 'tk_gps_data', 'tk_gps_time', 'tk_soundpressurelevel_spl', 'tk_humidity_humidity', 'tk_particulatematter_pm1_0', 'tk_particulatematter_pm2_5', 'tk_particulatematter_pm10_0', 'tk_dual0_20ma_solarlight', 'atmos_airtemperature', 'atmos_northwind', 'atmos_eastwind', 'atmos_gustwind']

column_labels = {
    'time': 'Time',
    'tk_gps_data': 'GPS_Data',
    'tk_gps_time': 'GPS_Time',
    'tk_soundpressurelevel_spl': 'Sound Pressure',
    'tk_humidity_humidity': 'Humidity',
    'tk_particulatematter_pm1_0': 'PM1.0',
    'tk_particulatematter_pm2_5': 'PM2.5',
    'tk_particulatematter_pm10_0': 'PM10',
    'tk_dual0_20ma_solarlight': 'Solar Light',
    'atmos_airtemperature': 'Air Temperature',
    'atmos_northwind': 'North_Wind',
    'atmos_eastwind': 'East_Wind',
    'atmos_gustwind': 'Gust_Wind'
}

# Initialize the data dictionary
data = {label: [] for label in column_labels.values()}
data['TGlobe'] = []

for feature in response:
    properties = feature['properties']
    for column in selected_columns:
        column_value = properties.get(column, None)
        data[column_labels[column]].append(column_value)
        if column == 'atmos_airtemperature':
            data['TGlobe'].append(column_value)

# Find the maximum length of lists in the dictionary
max_length = max(len(v) for v in data.values())

# Pad shorter lists with NaN values to make all lists of the same length
for key, value in data.items():
    if len(value) < max_length:
        data[key] = value + [np.nan] * (max_length - len(value))

# Create DataFrame from the modified dictionary
df = pd.DataFrame.from_dict(data)

print(df)

                      Time  GPS_Data    GPS_Time  Sound Pressure     Humidity  \
0     2023-06-19T15:21:10Z   60180.0  52912969.0      629.320000  5066.128713   
1     2023-06-19T15:21:11Z   60180.0  52913969.0      645.868687  5064.108911   
2     2023-06-19T15:21:12Z   60180.0  52914969.0      626.949495  5062.111111   
3     2023-06-19T15:21:13Z   60180.0  52915969.0      647.158416  5059.196078   
4     2023-06-19T15:21:14Z   60180.0  52916969.0      677.878788  5055.160000   
...                    ...       ...         ...             ...          ...   
1478  2023-06-19T15:45:48Z       NaN         NaN             NaN          NaN   
1479  2023-06-19T15:45:49Z       NaN         NaN             NaN          NaN   
1480  2023-06-19T15:45:50Z       NaN         NaN             NaN          NaN   
1481  2023-06-19T15:45:51Z       NaN         NaN             NaN          NaN   
1482  2023-06-19T15:45:52Z       NaN         NaN             NaN          NaN   

      PM1.0  PM2.5  PM10  S

In [69]:
# Extract latitude and longitude from features' geometry
latitude = []
longitude = []

for feature in response:
    geometry = feature['geometry']
    if geometry and 'coordinates' in geometry:
        coordinates = geometry['coordinates']
        long, lat, *_ = coordinates
        latitude.append(lat)
        longitude.append(long)

# Create a DataFrame for latitude and longitude
latitude_df = pd.DataFrame({'Latitude': latitude})
longitude_df = pd.DataFrame({'Longitude': longitude})


# Concatenate the latitude and longitude DataFrames with the existing DataFrame
df = pd.concat([df, latitude_df, longitude_df], axis=1)

In [70]:
df.columns.name = None
df.index.name = None

df

,Time,GPS_Data,GPS_Time,Sound Pressure,Humidity,PM1.0,PM2.5,PM10,Solar Light,Air Temperature,North_Wind,East_Wind,Gust_Wind,TGlobe,Latitude,Longitude
0,2023-06-19T15:21:10Z,60180.0,52912969.0,629.320000,5066.128713,3.0,4.0,4.0,88.039604,24.400000,-1.305,-0.015,1.310,24.400000,51.512188,-0.068554
1,2023-06-19T15:21:11Z,60180.0,52913969.0,645.868687,5064.108911,3.0,4.0,4.0,0.000000,24.400000,-1.560,0.430,1.620,24.400000,51.512187,-0.068555
2,2023-06-19T15:21:12Z,60180.0,52914969.0,626.949495,5062.111111,3.0,4.0,4.0,0.000000,24.299999,-1.735,0.575,1.840,24.299999,51.512187,-0.068555
3,2023-06-19T15:21:13Z,60180.0,52915969.0,647.158416,5059.196078,3.0,4.0,4.0,0.000000,24.200001,-0.960,0.280,1.230,24.200001,51.512186,-0.068556
4,2023-06-19T15:21:14Z,60180.0,52916969.0,677.878788,5055.160000,3.0,4.0,4.0,0.000000,24.200001,-1.765,-0.775,1.955,24.200001,51.512186,-0.068557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1478,2023-06-19T15:45:48Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.512197,-0.068492
1479,2023-06-19T15:45:49Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.512197,-0.068492
1480,2023-06-19T15:45:50Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.512196,-0.068491
1481,2023-06-19T15:45:51Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.512198,-0.068489


In [71]:
# Convert the 'time' column to datetime
df['Time'] = pd.to_datetime(df['Time'])
# Format the 'time' column as desired
df['Time'] = df['Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_all = df
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1483 entries, 0 to 1482
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Time             1483 non-null   object 
 1   GPS_Data         1464 non-null   float64
 2   GPS_Time         1464 non-null   float64
 3   Sound Pressure   1465 non-null   float64
 4   Humidity         1465 non-null   float64
 5   PM1.0            1465 non-null   float64
 6   PM2.5            1465 non-null   float64
 7   PM10             1465 non-null   float64
 8   Solar Light      1465 non-null   float64
 9   Air Temperature  1456 non-null   float64
 10  North_Wind       1456 non-null   float64
 11  East_Wind        1456 non-null   float64
 12  Gust_Wind        1456 non-null   float64
 13  TGlobe           1456 non-null   float64
 14  Latitude         1483 non-null   float64
 15  Longitude        1483 non-null   float64
dtypes: float64(15), object(1)
memory usage: 185.5+ KB


In [72]:
#ladybug load epw weather data
import math
from ladybug.epw import EPW

# Get altitude and longitude
from ladybug.location import Location
from ladybug.sunpath import Sunpath

# Create location. You can also extract location data from an epw file.
City = Location('LONDON/GATWICK', 'GBR', latitude=51.15, longitude=-0.18, time_zone=1)

# Initiate sunpath
sp = Sunpath.from_location(City)
sun = sp.calculate_sun(month=6, day=19, hour=15)

BerAlt = sun.altitude

#projection factor
def Pfactor(SunAlt):
    return 0.308*(math.cos(math.radians(SunAlt)*(1 - math.pow(math.radians(SunAlt),2) / 48402)))

ProF = Pfactor(BerAlt)

df_all['Wind Speed'] = np.sqrt(df['North_Wind']**2 + df['East_Wind']**2)
df_all['Humidity'] = df['Humidity']/100 
#df_all['Radition'] = 125*(df_all['Solar_Light']/1000000-4)


#MRT from Tglobe
D = 0.03 # globe diameter in meters
def mrt(Tg, Ta, W):
    return math.pow((math.pow((Tg + 273.15), 4)) + (1.1* math.pow(10, 8) * math.pow(W, 0.6) / (0.95 * math.pow(D, 0.4))) * (math.fabs(Tg - Ta)), 0.25) - 273.15

def MRT(mrt, Rad, PF):
    return math.pow((math.pow((mrt + 273), 4)) + ((abs(PF) * 0.7 * Rad) / (0.97 * 5.67 * math.pow(10, -8))), 0.25) - 273

#MRT from Tglobe
df_all['MRT'] = df_all.apply(lambda x: mrt(x['TGlobe'], x['Air Temperature'], x['Wind Speed']), axis=1)

#MRT from Tglobe + Solar Radiation
df_all['MRT_S'] = df_all.apply(lambda x: MRT(x['MRT'], x['Solar Light'], ProF), axis=1)                

In [73]:
from ladybug.epw import EPW
from ladybug_comfort.collection.utci import UTCI
import pandas as pd
from pythermalcomfort.models import utci

#df_all.loc[df_all['Gust_Wind'] < 0.5, 'Gust_Wind'] = 0.5

df_all['UTCI'] = df_all.apply(lambda x: utci(x['Air Temperature'], x['MRT'], x['Gust_Wind'], x['Humidity'], limit_inputs=False), axis=1)

df_all.info()

#pythermalcomfort.models.utci.utci(tdb,trv,rh,units='SI',return_stress_category=False,limit_inputs=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1483 entries, 0 to 1482
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Time             1483 non-null   object 
 1   GPS_Data         1464 non-null   float64
 2   GPS_Time         1464 non-null   float64
 3   Sound Pressure   1465 non-null   float64
 4   Humidity         1465 non-null   float64
 5   PM1.0            1465 non-null   float64
 6   PM2.5            1465 non-null   float64
 7   PM10             1465 non-null   float64
 8   Solar Light      1465 non-null   float64
 9   Air Temperature  1456 non-null   float64
 10  North_Wind       1456 non-null   float64
 11  East_Wind        1456 non-null   float64
 12  Gust_Wind        1456 non-null   float64
 13  TGlobe           1456 non-null   float64
 14  Latitude         1483 non-null   float64
 15  Longitude        1483 non-null   float64
 16  Wind Speed       1456 non-null   float64
 17  MRT           

In [74]:
df_all.to_csv('OutdoorWalks_London.csv')

In [75]:
def minMax(x):
    return pd.Series(index=['min','max'],data=[x.min(),x.max()])

df_all.apply(minMax)

,Time,GPS_Data,GPS_Time,Sound Pressure,Humidity,PM1.0,PM2.5,PM10,Solar Light,Air Temperature,North_Wind,East_Wind,Gust_Wind,TGlobe,Latitude,Longitude,Wind Speed,MRT,MRT_S,UTCI
min,2023-06-19 15:21:10,60180.0,52912969.0,539.14,48.16,2.0,2.01,2.01,0.000000,23.600000,-6.26,-4.93,0.09,23.600000,51.512126,-0.068562,0.007071,23.600000,23.600000,17.9
max,2023-06-19 15:45:52,60180.0,55336769.0,856.74,52.28,38.0,71.70,85.85,137.138614,25.950001,3.74,4.11,6.28,25.950001,51.515988,-0.061987,6.279148,25.950001,28.122934,25.5


In [76]:
import pandas as pd 
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import numpy as np

In [77]:
df_all = df_all.apply(pd.to_numeric, errors='coerce')

min_values = df_all.min(skipna=True)
max_values = df_all.max(skipna=True)

num_groups = 4
group_boundaries = {}
for column in df_all.columns:
    if pd.api.types.is_numeric_dtype(df_all[column]):  # Check if the column is numeric
        min_value = min_values[column]
        max_value = max_values[column]

        if np.isnan(min_value) or np.isnan(max_value):
            continue  # Skip columns with missing values

        group_size = (max_value - min_value) / num_groups
        if group_size > 0:
            boundaries = np.arange(min_value, max_value + group_size, group_size)
            group_boundaries[column] = boundaries

# Print the group boundaries for each numeric column
for column, boundaries in group_boundaries.items():
    print(f"Group boundaries for column '{column}':")
    print(boundaries)
    print()

Group boundaries for column 'GPS_Time':
[52912969. 53518919. 54124869. 54730819. 55336769.]

Group boundaries for column 'Sound Pressure':
[539.14 618.54 697.94 777.34 856.74]

Group boundaries for column 'Humidity':
[48.16 49.19 50.22 51.25 52.28]

Group boundaries for column 'PM1.0':
[ 2. 11. 20. 29. 38.]

Group boundaries for column 'PM2.5':
[ 2.01   19.4325 36.855  54.2775 71.7   ]

Group boundaries for column 'PM10':
[ 2.01 22.97 43.93 64.89 85.85]

Group boundaries for column 'Solar Light':
[  0.          34.28465347  68.56930693 102.8539604  137.13861386]

Group boundaries for column 'Air Temperature':
[23.60000038 24.18750048 24.77500057 25.36250067 25.95000076]

Group boundaries for column 'North_Wind':
[-6.26000023 -3.76000011 -1.25999999  1.24000013  3.74000025]

Group boundaries for column 'East_Wind':
[-4.92999983 -2.66999984 -0.40999985  1.85000014  4.11000013]

Group boundaries for column 'Gust_Wind':
[0.09       1.63750006 3.18500011 4.73250016 6.28000021]

Group bounda

In [78]:
import pandas as pd # data analysis package
import geopandas as gpd # extends the datatypes used by pandas to allow spatial operations on data
from shapely.geometry import Point # allows manipulation and analysis of points

import contextily as ctx # retrieves background tile maps from the internet
import requests # http library to retrieve maps from the internet

import matplotlib.pyplot as plt # library for creating static, animated, and interactive visualizations in Python

In [79]:
#interactive maps 

import os
import folium
from folium import plugins
from folium import Map, FeatureGroup, Marker, LayerControl
import branca.colormap as cm
import ipywidgets
import geocoder
import geopy
import numpy as np
import pandas as pd 

#import rioxarray as rxr

In [80]:
import sys
sys.path.insert(0, 'folium')
sys.path.insert(0, 'branca')
import branca
import folium 

In [81]:
from branca.element import MacroElement
from jinja2 import Template

class BindColormap(MacroElement):
    def __init__(self, layer, colormap):
        super(BindColormap, self).__init__()
        self.layer = layer
        self.colormap = colormap
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
            {{this._parent.get_name()}}.on('layeradd', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
                }});
            {{this._parent.get_name()}}.on('layerremove', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'none';
                }});
        {% endmacro %}
        """)    

In [82]:
#adding layers

locations = list(zip(df_all.Latitude, df_all.Longitude))

#function

def popup1(p): 
    p_str = str(round(p, 2))  # Convert p to a string and round it

    html = "<br>Value: " + p_str
    iframe = folium.IFrame(html)
    popup = folium.Popup(iframe, min_width=150, max_width=150)
    return popup


def circlemarker (layer, scale, feature_group):
    for loc, p in zip(zip(layer.iloc[:,0],layer.iloc[:,1]),layer.iloc[:,2]):
        folium.CircleMarker(
        color=scale(p),
        colormap=scale,
        location=loc,
        radius=2, 
        fill=True,
        popup=popup1(p)).add_to(feature_group)

In [84]:
# Sound_Pressure

sp = df_all[["Latitude", "Longitude", "Sound Pressure"]]
sp = sp.dropna(subset=['Sound Pressure'])

scale_fg1 = branca.colormap.StepColormap(['#D0B0D4','#A986B6','#835B99','#5C317B'], index=[539.14, 618.54, 697.94, 777.34, 856.74], vmin= 539.14, vmax = 856.74, caption ='Sound Pressure')
fg1 = FeatureGroup(name = 'Sound Pressure', overlay=True, control=False)
circlemarker(sp,scale_fg1,fg1)

#Humidity

humidity = df_all[["Latitude", "Longitude", "Humidity"]]
humidity = humidity.dropna(subset=['Humidity'])

scale_fg2 = branca.colormap.StepColormap(['#AEC7E7','#8397D7','#5766C7','#2C36B7'], index = [48.16, 49.19, 50.22, 51.25, 52.28], vmin=48.16, vmax=52.28, caption='Humidity')
fg2 = FeatureGroup(name ='Humidity', overlay=True, control=False)
circlemarker(humidity, scale_fg2, fg2)

#PM1_0

pm1 = df_all[["Latitude", "Longitude", "PM1.0"]]
pm1 = pm1.dropna(subset=['PM1.0'])
scale_fg3 = branca.colormap.StepColormap(['#00FFFF','#5AD3B1', '#E3D54D','#E86E51'], index = [2, 11, 20, 29, 38], vmin= 2, vmax = 38, caption='PM1.0')
fg3 = FeatureGroup(name = 'PM1.0', overlay=True, control=False)
circlemarker(pm1, scale_fg3, fg3)

#PM2_5 

pm25 = df_all[["Latitude", "Longitude", "PM2.5"]]
pm25 = pm25.dropna(subset=['PM2.5'])
scale_fg4 = branca.colormap.StepColormap(['#5AD3B1', '#E3D54D','#E86E51','#8C1E04'], index = [2.01,19.4325,36.855,54.2775,71.7], vmin= 2.01, vmax = 71.7, caption = 'PM2.5')
fg4 = FeatureGroup(name = 'PM2.5', overlay=True, control=False)
circlemarker(pm25, scale_fg4, fg4)

#PM10 

pm10 = df_all[["Latitude", "Longitude", "PM10"]]
pm10 = pm10.dropna(subset=['PM10'])
scale_fg5 = branca.colormap.StepColormap(['#5AD3B1', '#E3D54D','#E86E51','#8C1E04'], index=[2.01,22.97,43.93,64.89,85.85], vmin=2.01, vmax = 85.85, caption='PM10')
fg5 = FeatureGroup(name='PM10', overlay=True, control=False)
circlemarker(pm10, scale_fg5, fg5)


#Solar_Light

solar = df_all[["Latitude", "Longitude", "Solar Light"]]
solar = solar.dropna(subset=['Solar Light'])
scale_fg6 = branca.colormap.StepColormap(['#440154','#3b528b','#21918c','#5ec962'], index=[0, 34.28465347,68.56930693,102.8539604,137.13861386], vmin=0, vmax=137.14, caption='Solar Light')
fg6 = FeatureGroup(name='Solar Light', overlay=True, control=False)
circlemarker(solar, scale_fg6, fg6)

#Air_Temperature

air = df_all[["Latitude", "Longitude", "Air Temperature"]]
air = air.dropna(subset=['Air Temperature'])
scale_fg7 = branca.colormap.StepColormap(['#3DB063','#F2F50C','#F5950C','#FC0C00'], index=[23.60000038,24.18750048,24.77500057,25.36250067,25.95000076], vmin=23.60, vmax=25.95, caption='Air Temperature')
fg7 = FeatureGroup(name='Air Temperature', overlay=True, control=False)
circlemarker(solar,scale_fg7, fg7)

#MRT

mrt = df_all[["Latitude", "Longitude", "MRT"]]
mrt = mrt.dropna(subset=['MRT'])
scale_fg8 = branca.colormap.StepColormap(['#EE9A49','#CD853F','#8B5A2B','#2A1200'], index=[23.60000038, 24.18750048, 24.77500057, 25.36250067, 25.95000076], vmin=23.60, vmax=25.95, caption='MRT')
fg8 = FeatureGroup(name='MRT', overlay=True, control=False)
circlemarker(mrt, scale_fg8, fg8)


#MRT_S

mrt_s = df_all[["Latitude", "Longitude", "MRT_S"]]
mrt_s = mrt_s.dropna(subset=['MRT_S'])
scale_fg9 = branca.colormap.StepColormap(['#EE9A49','#CD853F','#8B5A2B','#2A1200'], index = [23.60000038, 24.73073374, 25.86146709, 26.99220045, 28.1229338], vmin =23.60, vmax = 28.12, caption = 'MRT*')
fg9 = FeatureGroup(name='MRT_S', overlay=True, control=False)
circlemarker(mrt_s, scale_fg9, fg9)

#UTCI 

utci = df_all[["Latitude", "Longitude", "UTCI"]]
utci = utci.dropna(subset=['UTCI'])
scale_fg10 = branca.colormap.StepColormap(['#00C000','#FF6600','#FF3300','#CC0000'], index = [17.9, 19.8, 21.7, 23.6, 25.5], vmin=17.9, vmax=25.5, caption='UTCI')
fg10 = FeatureGroup(name='UTCI', overlay=True, control=False)
circlemarker(utci, scale_fg10, fg10)

In [85]:
# Add each layer to the map and save it as a separate HTML file

for i in range(1, 11):
    # Create a new map object for each iteration
    m = folium.Map(location=[51.514810, -0.064867], control_scale=True, zoom_start=17, tiles=None)
    
    base_map1 = folium.FeatureGroup(name='OpenStreetMap', overlay=False)
    folium.TileLayer(tiles='OpenStreetMap').add_to(base_map1)
    # Add satellite imagery basemap
    base_map2 = folium.FeatureGroup(name="Esri World Imagery", overlay=False)
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Esri Satellite'
    ).add_to(base_map2)

    base_map1.add_to(m)
    base_map2.add_to(m)
    
    fg_name = "fg" + str(i)
    fg = globals()[fg_name]
    m.add_child(fg)
    
    scale_name = "scale_fg" + str(i)
    scale = globals()[scale_name]
    m.add_child(scale)
    m.add_child(BindColormap(fg, scale))

    # Add LayerControl to toggle between basemaps
    folium.LayerControl(position='bottomleft').add_to(m)
    
    map_fname = f'OutdoorWalk_London_Layer{i}.html'
    m.save(map_fname)